# Reinforcement Learning for CartPole

In this Lab, we will create a reinforcement learning-powered agent for the game of CartPole. Real-world analogies of the CartPole task (called inverted pendulums), like balancing a pencil on its tip, are [incredibly challenging](https://www.youtube.com/watch?v=U3vAoJhIWms). By using reinforcement learning, you should be able to train a neural network to learn to play this game automatically, and keep the inverted pendulum upright for 8 seconds without moving more than 2.4 units from its starting point.

Let's start by importing the relevant libraries. In this case, you're going to import TensorFlow and Python. Of course, with TensorFlow, you'll be able to accelerate Tensor operations - like matrix multiplications, which can be used to build fast neural networks. With the Python library, you'll be able to interoperate with Python libraries within Swift.

In [ ]:
import TensorFlow
import Python

Now, let's import the relevant Python libraries. The only ones you'll need for this lab are `numpy` and OpenAI's `gym`. To install them, though, we'll use the `os` module's `popen` function to run a `pip install` for the libraries we need. Then, we'll actually import them.

In [ ]:
let os = Python.import("os")
print(os.popen("pip install -U gym numpy").read())
let np = Python.import("numpy")
let gym = Python.import("gym")

Next, to make our lives easier, I'm defining a few utility functions. You don't need to understand the inner working of these functions - you just need to know what they do:

1. `makeTensor<T: NumpyScalarCompatible>() -> Tensor<T>` - this function will enable you to convert a numpy array to a Swift Tensor with a simple function call.
1. `load(weights: PythonObject)` - this function will enable you to load a numpy array as the weights of a neural network.
1. `exportWeights() -> PythonObject` - this function will export the weights of a neural network to a numpy array.

In case you'd like to figure out a bit more about how these functions work, here's a quick brief:

`makeTensor`

For `makeTensor` to work, we need to leverage a feature in Swift called "generics". With Generics, functions aren't bounded to returning or taking certain types - they're flexible, and can return or take any type. However, to make this feature useful, you can leverage "generic constraints", which confine which kinds of types can be passed to or returned by a function. While generics are present in many languages, Swift has a special implementation that preserves type information, while also providing another paradigm altogether for dynamic polymorphism.

In this case, we're declaring a new generic type called `T`, which must conform to the `NumpyScalarCompatible` protocol. Specifically, there's one part of this protocol that we're most interested in - the `numpyScalarTypes` property. This property is of type `[PythonObject]`, and essentially stores the numpy types of the corresponding Swift type. For example, Float's conformance to this protocol looks like this:

```
extension Float: NumpyScalarCompatible {
  public static let numpyScalarTypes: [PythonObject] = [np.float32]
  public static var ctype: PythonObject { return ctypes.c_float }
}
```

By leveraging this information, as well as Swift inferring the type of Tensor you wish to acquire, you can write the following code:

```
let x = np.random.random(5, 5, dtype="float64") // create a 5x5 random array of float64 type
let y: Tensor<Float> = a.makeTensor() // convert x to a Swift Tensor of 32-bit Float type
```

What happened is that because you explicitly defined the type of `y`, Swift knew that you wanted to convert to a floating-point Tensor, and told `makeTensor` to assume `T` is `Float`. To find out more about generics, [click here](https://docs.swift.org/swift-book/LanguageGuide/Generics.html).

`load(weights: PythonObject)`

For the `load` function to work, you need to leverage one of the most powerful features in Swift: key paths! Key paths enable you to iterate, recursively, through all the parameters in a data structure. Find out more about key paths [here](https://www.swiftbysundell.com/posts/the-power-of-key-paths-in-swift).

So, by iterating recursively through all the writable key paths in the neural network (`keyPath`), and keeping track of the index (`keyPathIndex`), I can load the weights from numpy into the network.

`exportWeights() -> PythonObject`

`exportWeights` kind of works like `load` in reverse - instead of keeping track of the index and loading weights, it simply appends the weights, one by one, in numpy form, to an array, which is then itself converted to a large numpy array and returned.

In [ ]:
extension PythonObject {
    func makeTensor<T: NumpyScalarCompatible>() -> Tensor<T> {
        return Tensor<T>(numpy: self.astype(T.numpyScalarTypes[0]))!
    }
}

extension Layer {
    mutating func load(weights: PythonObject) {
        for (keyPathIndex, keyPath) in self.recursivelyAllWritableKeyPaths(to: Tensor<Float>.self).enumerated() {
            self[keyPath: keyPath] = weights[keyPathIndex].makeTensor()
        }
    }
    
    func exportWeights() -> PythonObject {
        var weights: [PythonObject] = []
        for keyPath in self.recursivelyAllWritableKeyPaths(to: Tensor<Float>.self) {
            weights.append(self[keyPath: keyPath].makeNumpyArray())
        }
        return np.array(weights)
    }
}

Now, let's move on to the magic! _Neural networks!_

To begin, we need to define the neural network architecture. We'll call this network `CartPoleNetwork`. It's a structure that conforms to the `Layer` protocol, which is shipped inside the Swift for TensorFlow standard library. There are a few conditions that you must meet to conform to the `Layer` protocol:

1. You must define the type of the `Input` to your layer.
1. You must define the type of the `Output` of your layer.
1. You must define a function with signature `callAsFunction(_ input: Input) -> Output`

With the third constraint you'll be able to do things like this:

```
let network = CartPoleNetwork()
let data: Tensor<Float> = ... // make some data somehow
let output = network(data)
```

On line 3, you passed data to the network as if it was a function - but it's not. You're calling a structure as if it was a function; but actually, the above is the same as doing:

```
let network = CartPoleNetwork()
let data: Tensor<Float> = ... // make some data somehow
let output = network.callAsFunction(data)
```

It's basically just syntactic sugar.

The neural network that will play CartPole is very simple - it has an input layer with 4 units, a hidden layer with 128 units, and an output layer with 2 units. The first layer's activation is ReLU (Rectified Linear Unit, a clip operation with `(min: 0, max: inf)`), and the second layer's activation is Softmax (log probability).

Inside the `callAsFunction` function, we're simply going to run the input through the 2 Dense layers. There are two ways of doing this:

1. `l2(l1(input))`
2. `input.sequenced(through: l1, l2)`

For smaller networks, even the first one is okay. But for good practice, we'll use the second one.

In [ ]:
struct CartPoleNetwork: Layer {
    // Define the types of input & output for this Layer.
    typealias Input = Tensor<Float>
    typealias Output = Tensor<Float>

    // Declare the layers within this layer.
    var l1, l2: Dense<Float>

    init() {
        // Define the layers within this layer.
        l1 = Dense(inputSize: 4, outputSize: 128, activation: relu)
        l2 = Dense(inputSize: 128, outputSize: 2, activation: softmax)
    }

    // Define the differetiable function that represents the forward-pass of this layer.
    @differentiable
    func callAsFunction(_ input: Input) -> Output {
        return input.sequenced(through: l1, l2) // Sequence the input through the nested layers.
    }
}

Now, you may not realize this - but line 12 is the most powerful line in this code snippet. The `@differentiable` function decorator is the most powerful line in this code block. But why is a decorator so special?

Well, this decorator is, in a way, the root cause for developing Swift for TensorFlow. If you're a programmer that enjoys manually calculaing and defining derivatives & gradients for complex neural networks, then S4TF isn't for you. That's because S4TF has a very powerful Autodiff (Automatic Differentiation) system, which enables a simple function decorator to enable S4TF to calculate the gradient of a function with respect to all of its differentiable parameters - including `self`, since in this case, the function is inside of a `struct`.

Soon, it'll support all sorts of control flow, and more. It already supports `if` statements!

Next, we need to actually train the neural network. Because there's no labelled training data, we need to generate some ourselves. To do so, let's start off by defining what an "episode" of gameplay looks like. Specifically, the fact that it contains "steps" and the episode has a "reward", which is how well the episode went. Each step consists of a game state represented as a Tensor, and the action that the player (neural network) decided to make when confronted with that state.

In [ ]:
struct Episode {
    struct Step {
        let state: Tensor<Float> // The current game state.
        let action: Int32 // The action that was decided for the game state.
    }

    let steps: [Step] // A list of state->action mappings.
    let reward: Float // Episode reward (i.e. "how good was this episode?").
}

Next, there needs to be some way to generate a lot of gameplay episodes with which the neural network can be trained. To streamline this task, let's make a function that does it for us! It'll take an OpenAI Gym Environment, the number of episodes to create (the number of games to play), and the neural network with which to make decisions. It'll return an array of episodes.

In [ ]:
func generateEpisodes(env: PythonObject, episodes episodeCount: Int, network: CartPoleNetwork) -> [Episode] {
    var episodes: [Episode] = []

    for _ in 1...episodeCount {
        var currentState: Tensor<Float> = env.reset().makeTensor() // Get the initial state's tensor
        var steps: [Episode.Step] = []
        var episodeReward: Float = 0

        while true {
            let actionProbabilities = network(currentState.rankLifted())[0].makeNumpyArray() // Get neural network analysis of state

            let action = np.random.choice(2, p: actionProbabilities) // Choose action with probabilities from neural network
            let (nextState, reward, isDone, _) = env.step(action).tuple4 /* Apply the action to the environment, and get
                                                                            the next state, the reward, and whether the game is finished */

            steps.append(Episode.Step(state: currentState, action: Int32(action)!))

            episodeReward += Float(reward)!

            if isDone == true {
                episodes.append(Episode(steps: steps, reward: episodeReward)) // If the game is done, store the episode ...
                break // and break the infinite loop!
            }

            currentState = nextState.makeTensor() // If game is still going, update the currentState Tensor
        }
    }

    return episodes
}

Finally, to make the training process simpler, there has to be a function that discards bad episodes, and keeps the good episodes that happened by chance / with luck. Over time, these episodes will rely less and less on luck, and more and more on skill.

To do this, all the function needs to do is take an array of episodes, find the 70th percentile of the rewards, and only keep episodes whose rewards are higher than that 70th percentile. Then, do a simple processing task on those episodes to make the data shape suitable for the neural network.

In [ ]:
func preprocessEpisodes(episodes: [Episode]) -> (input: Tensor<Float>, target: Tensor<Float>, rewards: Tensor<Float>, meanReward: Float) {
    let epRewards = episodes.map { $0.reward }
    let rewardBound = Float(np.percentile(epRewards, 70))! // Find the 70th percentile of the rewards
    print("Current reward boundary: \(rewardBound)")

    let validEpisodes = episodes.filter({ $0.reward >= rewardBound }) // Filter out the bad episodes
    let meanReward = validEpisodes.map({ $0.reward }).reduce(0, +) / Float(validEpisodes.count) // Find the current mean reward
    print("Mean reward: \(meanReward)")
    print("Valid episodes: \(validEpisodes.count)")

    var input: Tensor<Float> = Tensor(0)
    var target: Tensor<Float> = Tensor(0)
    var rewards: Tensor<Float> = Tensor(0)

    // Take all states/actions/rewards from the valid episodes and put them into one Tensor each
    for (episodeIndex, episode) in validEpisodes.enumerated() {
        let states = Tensor<Float>(episode.steps.map { $0.state })
        let actions = Tensor<Float>(oneHotAtIndices: Tensor<Int32>(episode.steps.map { $0.action }), depth: 2)
        let reward = Tensor<Float>(ones: [actions.shape[0], 1]) * Tensor<Float>(episode.reward)

        if episodeIndex == 0 {
            input = states
            target = actions
            rewards = reward
        } else {
            input = input.concatenated(with: states)
            target = target.concatenated(with: actions)
            rewards = rewards.concatenated(with: reward)
        }
    }

    // Return the training data
    return (input, target, rewards, meanReward)
}

And... we're ready to train! Doing so is surprisingly easy.

In essence, you're going to initialize your neural network and an optimizer - in this case, the Adam optimizer. You're then going to create your Gym environment and create a cache for the rewards of previous episodes.

The maximum reward one can get in the game of CartPole is 500 - after which the game is considered "won". So, if 5 batches of episodes consecutively come back with a mean reward of 500, the training process stops.

Inside the loop, we generate 20 episodes using the current neural network, and then process them to only keep the cream of the crop. Then, the neural network is trained by finding the gradient with respect to its parameters through the Softmax Cross-Entropy loss function, evaluating it against the moves it made itself, multiplied by the reward of the episode that the samples were a part of. The optimizer updates the neural network's weights, the mean reward for these episodes is used to update the cache, and the cycle continues.

In [ ]:
var network = CartPoleNetwork()
var optimizer = Adam(for: network, learningRate: 0.008)

let env = gym.make("CartPole-v1")
var meanRewardCache: [Float] = [0, 0, 0, 0, 0]

while meanRewardCache.map({ $0 >= 500 }).contains(false) {
    let episodes = generateEpisodes(env: env, episodes: 20, network: network) // Generate a batch of episodes
    let (input, target, rewards, batchMeanReward) = preprocessEpisodes(episodes: episodes) // Preprocess the batch of episodes

    let gradients = network.gradient { network -> Tensor<Float> in //
        let p = network(input)                                     //
        let l1 = target                                            //
        let l2 = log(p)                                            // Get the gradient of this function
        let l3 = (1 - target)                                      // with respect to the parameters in
        let l4 = log(1 - p)                                        // the "network".
        let loss = -((l1 * l2 + l3 * l4) * rewards).mean()         // This function calculates
        print("Loss: \(loss)")                                     // Softmax Cross-Entropy loss.
        return loss                                                //
    }                                                              //
    optimizer.update(&network.allDifferentiableVariables, along: gradients) // Update the network along the gradients

    meanRewardCache.append(batchMeanReward) // Cache the mean episode reward
    meanRewardCache.removeFirst() // Remove the first reward value in the cache (keep it at length 5)
}

print("Solved!") // The problem is solved!

Now that you've trained the neural network, it's time to visualize your results! What better way than to actually view the neural net playing Cartpole?

Remember, you're running this in a Jupyter Notebook. Therefore, there's no display that OpenAI Gym can use to render the cartpole game. So, for this to work without you actually running the code yourself locally, we'll need to use a small workaround with a package named `pyvirtualdisplay`.

**NOTE: This code will NOT work locally on your own machine. Refer 2 cells down for code that'll work for you locally!**

The following cell of code will do a few things:
1. Install the required packages to render the Gym environment.
1. Create and start the virtual display in which the environment will be rendered.
1. Create the Cartpole environment and wrap it in a "Monitor" wrapper, which will force Gym to render the game to a video file, instead of to the display directly.
1. Play the game, taking the argmax of the final layer as the action for the current state.
1. Continue step 4 until the game is over.

In [ ]:
print(os.popen("conda install -c conda-forge ffmpeg -y").read())
print(os.popen("pip install pyvirtualdisplay").read())
let Display = Python.import("pyvirtualdisplay").Display

let display = Display(visible: 0, size: [1400, 900])
display.start()

let env = gym.wrappers.Monitor(gym.make("CartPole-v1"), "/resources/cartpole-rl-results", force: true)
var state: Tensor<Float> = env.reset().makeTensor()
while true {
    env.render()
    let action = network(state.rankLifted())[0].argmax().scalar!
    let (nextState, _, isDone, _) = env.step(action).tuple4
    if isDone == true {
        env.close()
        break
    } else {
        state = nextState.makeTensor()
    }
}

Hooray! The above cell would have created a video file that we can now display. Let's do that here:

**NOTE: This code will NOT work locally on your own machine. Refer 1 cell down for code that'll work for you locally!**

In [ ]:
print(os.popen("wget -N https://github.com/google/swift-jupyter/raw/master/EnableIPythonDisplay.swift").read())
%include "EnableIPythonDisplay.swift"
let Video = Python.import("IPython.display").Video
let filename = "/resources/cartpole-rl-results/openaigym.video.\(env.file_infix).video000000.mp4"
print(filename)
Video(filename, width: 600, embed: true).display()

If you're running this code locally, and would like to see your neural network playing Cartpole, feel free to run the following code, without the first and last lines (they put the code block in a comment, so you don't run them in the notebook):

In [ ]:
/*
let env = gym.make("CartPole-v1")
var state: Tensor<Float> = env.reset().makeTensor()
while true {
    env.render()
    let action = network(state.rankLifted())[0].argmax().scalar!
    let (nextState, _, isDone, _) = env.step(action).tuple4
    if isDone == true {
        env.close()
        break
    } else {
        state = nextState.makeTensor()
    }
}
*/

<hr>
<h2>The last exercise!</h2>
<p>Congratulations, you have completed your first lesson and hands-on lab in Python. However, there is one more thing you need to do. The Data Science community encourages sharing work. The best way to share and showcase your work is to share it on GitHub. By sharing your notebook on GitHub you are not only building your reputation with fellow data scientists, but you can also show it off when applying for a job. Even though this was your first piece of work, it is never too early to start building good habits. So, please read and follow <a href=“https://cognitiveclass.ai/blog/data-scientists-stand-out-by-sharing-your-notebooks/” target=“_blank”>this article</a> to learn how to share your work.
<hr>